In [1]:
from sympy import symbols, init_printing
import sympy
import sympy.physics.mechanics as me
init_printing(use_latex='mathjax')
import seaborn as sns
# sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
import matplotlib.pyplot as plt
from scipy.integrate import ode, odeint
from scipy.linalg import eigvals
import numpy as np
from numpy import sqrt
%matplotlib inline
import InputShaping as shaping
from scipy import linalg

//anaconda/envs/py2p7/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
# Create the variables
x, y, beta = me.dynamicsymbols('x, y, beta')

# Create the velocities
x_dot, y_dot, beta_dot = me.dynamicsymbols('x, y, beta', 1)

# Create the constants
m, k, L, g, H, c, D,t, Izz, k_beta, c_beta = sympy.symbols('m k L g H c D t Izz k_beta c_beta')
L_1_init, L_2_init = sympy.symbols('L_1_init L_2_init')
# m = mass
# k = spring k
# L = spring equilibrium length
# g = gravity
# c = spring c
# c_beta = rotational c
# k_beta = rotational k
# D = rod length
# Izz = moment of Inertia about the end of a rod

In [3]:
# Create the world frame
N = me.ReferenceFrame('N')

# Create the rod frame
B = N.orientnew('B', 'axis', [beta, N.z])

# Set the rotation of the rod frame
B.set_ang_vel(N, beta_dot * N.z)

# Create the Origin
O1 = me.Point('O_1')

# Set origin velocity to zero
O1.set_vel(N, 0 * N.x)

# Create the second attachment point
O2 = O1.locatenew('O_2', H * N.x)
O2.set_vel(N, 0 * N.x)

# Locate the point in the N frame
# P = me.Point('pen')
P = O1.locatenew('P', x * N.x + y * N.y)

# P.set_pos(O1, x * N.x + y * N.y)

# Set the point's velocity
P.set_vel(N, x_dot * N.x + y_dot * N.y)

# Create the rod center of mass
G = P.locatenew('G', D/2 * B.y)

# Set the velocity of G
G.v2pt_theory(P, N, B)

# Create the rod
I_rod = me.inertia(B, 0, 0, Izz)
rod = me.RigidBody('rod', G, B, m, (I_rod, G))

# Create the distance from the point to each attachment point
L1 = O1.pos_from(P).magnitude
L2 = O2.pos_from(P).magnitude
L1_vector = O1.pos_from(P).normalize
L2_vector = O2.pos_from(P).normalize

# Create the height from the center of gravity to the datum
h = G.pos_from(O1) & N.y

# The forces at the connection point
forceP = c * (x_dot + y_dot) * L1_vector() + c * (x_dot + y_dot) * L2_vector()

# The forces on the beta frame
forceB = -c_beta * beta_dot * N.z

rod.potential_energy = (-m * g * h + 0.5 * k * (L1() - L_1_init)**2 + 0.5 * k *(L2() - 
                        L_2_init)**2 + 0.5 * k_beta * beta**2)

Lag = me.Lagrangian(N, rod)

In [4]:
LM = me.LagrangesMethod(Lag, [x, y, beta], forcelist=[(P, forceP), (B, forceB)], frame=N)

# No damping
# LM = me.LagrangesMethod(Lag, [x, y, beta], frame=N)
EqMotion = LM.form_lagranges_equations()
lrhs = LM.rhs()

In [5]:
operating_point = {x:10, y:10, beta:0, x_dot:0, y_dot:0, beta_dot:0}

A, B, inp_vec = LM.linearize([x,y,beta], [x_dot, y_dot, beta_dot],
                             op_point=operating_point,
                             A_and_B=True)

In [6]:
A_Sub = A.subs({k:1.0,m:1.0,g:9.8, L_1_init:14.8358073760749,L_2_init:14.8358073760749,H:20.0,c:1.0,c_beta:1.0,x_dot:0.0,
        y_dot:0.0, beta_dot:0.0, D:1.0,Izz:0.33,k_beta:1.0})
test = A_Sub.doit()

In [10]:
print(test)

Matrix([[0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1], [-1.71447121212122 - 0.0439393939393939*sqrt(2)*(-14.8358073760749 + 10*sqrt(2)), 0.0439393939393939*sqrt(2)*(-14.8358073760749 + 10*sqrt(2)) + 0.0431045454545421, -8.93939393939394, -0.878787878787879*sqrt(2) + 1.2427937366309, -0.878787878787879*sqrt(2) + 1.2427937366309, -1.51515151515152], [0.025*sqrt(2)*(-14.8358073760749 + 10*sqrt(2)) + 0.0245249999999981, -0.975475000000002 - 0.025*sqrt(2)*(-14.8358073760749 + 10*sqrt(2)), 0, -0.5*sqrt(2) - 0.707106781186547, -0.5*sqrt(2) - 0.707106781186547, 0], [-1.47799242424243 - 0.0378787878787879*sqrt(2)*(-14.8358073760749 + 10*sqrt(2)), 0.0378787878787879*sqrt(2)*(-14.8358073760749 + 10*sqrt(2)) + 0.037159090909088, -17.8787878787879, -0.757575757575758*sqrt(2) + 1.07137391088871, -0.757575757575758*sqrt(2) + 1.07137391088871, -3.03030303030303]])


In [7]:
eignenvalues = eigvals(test)
eigen = np.sqrt(np.abs(np.unique(np.real(eignenvalues))))*(1.0/2*np.pi)
eigen_index = np.argsort(eigen)
high, medium, low = eigen[eigen_index]

In [8]:
print(high,medium,low)

(0.089256331073574655, 1.3208770002955303, 1.931455363949018)
